# Setup data

In [8]:
devtools:::load_all("../occcompaRe")
devtools:::load_all(".")

source("R_data_preparation//helper_dataPreparation.R")
ls()

Loading occCompare
Loading occCompareExp


[1] "addForest"                           
[2] "dPrep_agri_cult1_to_L1_class_mapping"
[3] "dPrep_get_ids_orig"                  
[4] "dPrep_get_newUold_ids"               
[5] "dPrep_pixel_poly_stats"              
[6] "dPrep_reclassify"                    
[7] "dPrep_scale_minmax"                  
[8] "q"                                   
[9] "quit"

## Reference Data

Load the original reference data (this also creates the file "Kulturen_Ids_nPix_mainTypes.txt"):

In [9]:
addForest=TRUE
orig <- dPrep_get_ids_orig(dir="../data/raster", buf=20, rmUn=FALSE, addForest=addForest)

Create a mapping data frame (lut_reclass) between the original IDs and the main crop types that can be used for reclassification:

In [10]:
lut_origUtypes <- read.csv("data/raster/Kulturen_Ids_nPix_mainTypes.txt")
lut_origUtypes

X,Nutzungen,Code,type,nPix
1,Sonstiges,3,others,0
2,Koernermais,10,corn,2374
3,Winterfutterweizen,11,wcereals,58769
4,Winterfuttergerste,12,wcereals,42988
5,Winterfutterroggen,13,wcereals,7489
6,Sommerhafer,14,scereals,9860
7,Wintertriticale,15,wcereals,39713
8,sonstigesGetreide,16,others,683
9,Silomais,17,corn,145260
10,Wintermenggetreide,18,wcereals,1383


In [11]:
lut_origUtypes <- lut_origUtypes[lut_origUtypes$nPix!=0, ]
# IDs/Labels
classnames <- c("unlabeled", "corn", "oilseeds", "pgrass", 
                "root", "scereals", "wcereals", "bforest", "cforest", "others")
lut_reclass <- data.frame(id=0:(length(classnames)-1), 
                  name=classnames, oldId=NA)
lut_reclass

id,name,oldId
0,unlabeled,NA
1,corn,NA
2,oilseeds,NA
3,pgrass,NA
4,root,NA
5,scereals,NA
6,wcereals,NA
7,bforest,NA
8,cforest,NA
9,others,NA


In [12]:
for (i in 1:nrow(lut_reclass))
  lut_reclass[i, "oldId"] <- paste(sort(
    lut_origUtypes$Code[lut_origUtypes$type==lut_reclass$name[i]]),
    collapse=",")
lut_reclass

id,name,oldId
0,unlabeled,"87,203,206"
1,corn,"10,17"
2,oilseeds,"21,22,23"
3,pgrass,"75,77,93"
4,root,"61,67"
5,scereals,"14,26,33,34,36,37,38"
6,wcereals,"11,12,13,15,18,24,27"
7,bforest,241
8,cforest,242
9,others,"16,31,32,35,43,57,62,63,64,70,71,72,73,74,76,83,85,175,185,202,204,205"


Reclassify the original ids to the new classes:

In [13]:
ref <- list(ids=dPrep_reclassify(orig$ids, lut=lut_reclass))
ref$cells <- orig$cells
ref$freq <- table(ref$ids)
ref$lut <- lut_reclass[, 1:2]

Get the field information:

In [14]:
# Format data as required for experiments
ref$fids <- raster("data/raster/agri_fid")[ref$cells]
ref$fids[ref$ids==0] <- 0
fidsize <- as.numeric(table(ref$fids))
filen <- lapply(ref$lut$id, function(i) {
  tbl <- table(ref$fids[ref$ids==i])
  list(ufids=as.numeric(names(tbl)),
       len=as.numeric(tbl))})
class_stats <- data.frame(name=ref$lut$name, 
                          id=ref$lut$id,
                          nSmpls=sapply(filen, function(x) sum(x$len)), 
                          nFlds=sapply(filen, function(x) length(x$len)))
class_stats

name,id,nSmpls,nFlds
unlabeled,0,4762912,1
corn,1,147634,1336
oilseeds,2,39816,315
pgrass,3,469643,5685
root,4,8722,87
scereals,5,48701,535
wcereals,6,201480,1934
bforest,7,7721,1
cforest,8,7750,1
others,9,172989,2028


## Image Data

We used four RapidEye (RE) acquisitions and TerraSAR-X (TSX) acquisitions from 2011.

In [15]:
re20 <- brick("data/raster/re_20bands")[]
re20 <- re20[ref$cells, ]; gc(verbose=FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1038658,55.5,3654118,195.2,7250569,387.3
Vcells,82323614,628.1,375783573,2867.1,469326403,3580.7


In [16]:
tsx <- brick("data/raster/tsx139")[]
tsx <- tsx[ref$cells, ]; gc(verbose=FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1038947,55.5,3654118,195.2,7250569,387.3
Vcells,152732196,1165.3,433043475,3303.9,489421122,3734.0


In [17]:
re20Utsx <- cbind(re20, tsx); gc(verbose=FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1038929,55.5,3654118,195.2,7250569,387.3
Vcells,340487867,2597.8,519732170,3965.3,489421122,3734.0


In [18]:
colnames(re20Utsx)

[1] "re_062_0303_B"      "re_062_0303_G"      "re_062_0303_R"     
 [4] "re_062_0303_RE"     "re_062_0303_IR"     "re_092_0402_B"     
 [7] "re_092_0402_G"      "re_092_0402_R"      "re_092_0402_RE"    
[10] "re_092_0402_IR"     "re_178_0627_B"      "re_178_0627_G"     
[13] "re_178_0627_R"      "re_178_0627_RE"     "re_178_0627_IR"    
[16] "re_268_0925_B"      "re_268_0925_G"      "re_268_0925_R"     
[19] "re_268_0925_RE"     "re_268_0925_IR"     "tsx139_133_0513_VH"
[22] "tsx139_166_0615_VH" "tsx139_199_0718_VH" "tsx139_232_0820_VH"
[25] "tsx139_254_0911_VH" "tsx139_265_0922_VH" "tsx139_133_0513_VV"
[28] "tsx139_166_0615_VV" "tsx139_199_0718_VV" "tsx139_232_0820_VV"
[31] "tsx139_254_0911_VV" "tsx139_265_0922_VV"

Check/fin seeds such that min. size of samples per class >= 1500 samples:

In [19]:
# ----------------------------------------------------------
# TRAIN, TEST DATA SPLIT SEEDS
nMin <- 1500
pTr <- 0.5
seeds <- numeric(100)
seed <- 0
cat("Adding seeds: ")

Adding seeds: 

In [ ]:
while (any(seeds==0)) {
  seed <- seed+1
  tbl <- table(ref$ids[get_tr_fids_idx(ref$ids, ref$fids, pTr, seed=seed)])
  if (min(tbl)>=nMin) {
    cat(seed, ".")
    seeds[which(seeds==0)[1]] <- seed
  }
}

Save all the data required for running the experiments:

In [29]:
outdir <- "data/rdata_agri6cl_tryedToReprod"
if (addForest)
    outdir <- paste0(outdir, "Uforest")
dir.create(outdir)
# ----------------------------------------------------------
# Save
saveRDS(ref$cells, paste0(outdir, "/cells.rds"))
saveRDS(ref$fids, paste0(outdir, "/fids.rds"))
saveRDS(fidsize, paste0(outdir, "/fidsize.rds"))
saveRDS(filen, paste0(outdir, "/filen.rds"))
saveRDS(ref$ids, paste0(outdir, "/ids.rds"))
saveRDS(re20, paste0(outdir, "/re.rds"))
saveRDS(tsx, paste0(outdir, "/tsx.rds"))
saveRDS(re20Utsx, paste0(outdir, "/reUtsx.rds"))
saveRDS(seeds, paste0(outdir, "/seeds.rds"))
saveRDS(class_stats, paste0(outdir, "/class_stats.rds"))

Warning message:
In dir.create(outdir): 'data\rdata_agri6clUforest' existiert bereits

In [30]:
class_stats # here

name,id,nSmpls,nFlds
unlabeled,0,4762912,1
corn,1,147634,1336
oilseeds,2,39816,315
pgrass,3,469643,5685
root,4,8722,87
scereals,5,48701,535
wcereals,6,201480,1934
bforest,7,7721,1
cforest,8,7750,1
others,9,172989,2028
